In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [2]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [3]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [4]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'):
        if not 'Primera' in info.get_text(): #Exclude 'Primera Publicacion'
            if not 'Contrato' in info.get_text():
                anuncio_item = info.get_text()
                anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
                anuncio_list.append(anuncio_item.replace('\n',''))  #Get items without \n
                #anuncio_list.append(info.get_text().replace('\n',''))
    return anuncio_list

In [5]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [6]:
def get_name_empresa(result_page):
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    return Enterprise_information.find('strong').get_text()

In [7]:
def clean_text(text):
    import unidecode
    text = text.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [8]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [9]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
description_list = get_job_description(result_page)
empresa_name = get_name_empresa(result_page)
output = unify_lists([empresa_name, anuncio_list, description_list])

In [10]:
output

['NOSSAMODA SRL',
 '695472',
 'Santa Cruz de la Sierra, Bolivia',
 'Contabilidad y Economía ',
 'De Bs. 3.000 a Bs. 3.999 ',
 '28 Enero 2019',
 '27 Febrero 2019',
 'analista de tesoreria formacion titulado en auditoria financiera contaduria publica contaduria general o ramas afines experiencia y conocimientos  tres  anos de experiencia laboral en cargos de igual responsabilidad  elaboracion de conciliaciones bancarias y arqueos de caja elaboracion de flujos de caja gestion de pagos a proveedores locales y extranjeros seguimiento de cuentas pendientes de cobro y depositos de cobranzas del dia  preparacion de estados de cuentas por cobrar y pagar y control de emision de cheques registro contable de pagos a proveedores sueldos finiquitos etc  registro contable de cobros a clientes depositos transferencias etc  conocimiento en sistema contable sai   solidos manejo de windows office excel powerpoint outlook y word competencias  responsabilidad  organizacion  atencion a los detalles las pers

In [11]:
def trabajopolis(job_description):
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    for job in job_description:
        #url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job + "&page=1&listings_per_page=1000"
        url = "https://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D=cajero&page=1&listings_per_page=10"

        response = requests.get(url)
        if response.status_code == 200:
            print("The request response cycle was successful")
            results_page = BeautifulSoup(response.content,'lxml')
            #trabajo_list = list()
            trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
            for anuncio in trabajos:
                trabajo_link = anuncio.find('a').get('href')
                trabajo_name = anuncio.find('b').get_text()
                print(trabajo_link)
             #   trabajo_list.append((trabajo_name,trabajo_link))
            #for trabajo in trabajo_list:
                output = get_job_info(trabajo_link)
                trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
        else:
            print("The request response cycle work didn't work")
        time.sleep(10) 
    return trabajos_list

In [12]:
def trabajopolis2():
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.trabajopolis.bo/search-results-jobs/?searchId=1546400657&action=search&page=1&listings_per_page=1500"
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_name)
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [13]:
Results = trabajopolis2()

The request response cycle was successful
Responsable de Recursos Humanos - Uyuni
Jefe de Producción
Auxiliar Contable Administrativo(a)
Analista de Datos - La Paz
Vendedor(a) de Equipos e Insumos Médicos – Hospitalarios
Asesoras(es) Ejecutivas(os)
Cajero(a)
Cobrador(a) de Calle
Ejecutivo(a) de Ventas - Especializado(a) en Rubro Ferretero - Santa Cruz
Chofer
Jefe de Área Legal
Auxiliar Contable
Key Account - Supermercado - Santa Cruz
Jefe de Contabilidad - La Paz
Supervisor(a) de Marketing - Santa Cruz
Ejecutivo(a) de Ventas - Línea Baterías - Santa Cruz
Coordinador(a) de Sucursal - Quillacollo
Diseñador(a) Gráfico(a)
Asesor(a) Comercial
Ejecutivo(a) de Ventas
Supervisor(a) de Almacén - Cochabamba
Asistente de Auditoría
Supervisor(a) Operativo(a)
Analista de Información e Investigación - La Paz
Motoquero
Jefe de Agencia - Aiquile
Encargado(a) de Caja - El Alto
Jefe de Agencia (Plazo Fijo) - La Paz
Asesor(a) de Cobranza y Recuperaciones - Cochabamba (Plazo Fijo)
Asesor(a) PyME - Santa C

Vendedor(a)
Vendedor(a) - La Paz
Encargado(a) de Seguridad, Salud Ocupacional y Medio Ambiente - El Alto
Supervisor(a) de Ventas
Pasante para el Área Comercial - La Paz
Cajero(a) - Apoyo Administrativo
Asistente Comercial - Santa Cruz
Auxiliar Contable Administrativo(a)
Ejecutivo(a) de Ventas AUDI - Cochabamba
Asistente de Logística
Transcriptor(a) - Apoyo Administrativo
Encargado(a) de RR.HH.
Supervisor(a) de Administración de Personas
Jefe de Modelado y Electricidad
Supervisor(a) de Cartera y Cobranzas
Asistente de Promoción Educativa
Programador(a)
Cajero(a) y Atención al Cliente
Auxiliar Administrativo(a) Financiero(a)
Auxiliar de Sistemas
Encargado(a) de Ventas - División Aditivos
Promotor(a)
Técnicos FTTX - Santa Cruz
Auxiliar Contable
Personal para Ventas
Asesor(a) Comercial - División Camiones - Cochabamba
Consultor(a) de Ventas Corporativas
Comercio Exterior - La Paz
Técnicos de Mantenimiento de Fibra Óptica (3 Vacantes)
Trade Marketing
Gestor(a) de Recursos Humanos - Santa Cr

Pasante para el Área de Compras
Asistente Contable Administrativo(a) - Santa Cruz
Asistente de Transporte y Distribución - El Alto
Supervisor(a) de Transporte y Distribución - El Alto
Auxiliar Administrativo(a) Regional - Cochabamba
Responsable de Proyecto “Reorganización del Control de Almacenes”
Servicio al Cliente - Operaciones
Asesor(a) Comercial Freelance - Santa Cruz
Sub Gerente de Agencia (Tarija – Bermejo)
Asesor(a) de Ventas – Maquinaria Pesada - Santa Cruz
Asesor(a) de Crédito Individual - Tarija
Coordinador(a) Digital - Santa Cruz
Lic. en Enfermería para Representante Comercial
Analista de Gestión Humana - Santa Cruz
Jefe de Marketing - Santa Cruz
Jefe de Tesorería - Temporal - Santa Cruz
Asesor(a) Legal
Cajeros(as) - Santa Cruz
Contador(a) General
Ejecutivos(as) de Ventas Técnicas Comerciales
Analista de Desarrollo y Mantenimiento Web - La Paz
Trade Marketing - Masivos (Santa Cruz)
Jefe de Distribución - Santa Cruz
Ejecutivo(a) de Ventas Corporativas
Arquitecto(a) Técnico C

Asesor(a) Legal - Santa Cruz
Analista de Inteligencia de Negocios - Santa Cruz
Analista de Innovación - Santa Cruz
Analista de Desarrollo Tecnológico - Santa Cruz
Jefe de Trade Marketing
Experto(a) en Autoridad del Juego - Promociones Empresariales
Recepcionista
Pasante de Soporte Informático
Analista Comercial de Posgrado
Ejecutivo(a) Comercial - La Paz
Asistente de Proyectos
Coordinador(a) Comercial - La Paz
Asistente de Compras
Ejecutivo(a) de Ventas – Canal Institucional y Eventos
Analista de Riesgo de Crédito
Analista de Cuentas Especiales
Ejecutivo(a) de Ventas/Negocios - Área Seguridad Industrial
Encargado(a) de Plataforma
Oficial de Créditos Júnior
Oficial de Créditos Semi Sénior
Oficial de Créditos Júnior
Jefe de Almacén - Santa Cruz
Visitador(a) Médico(a)
Ingeniero(a) de Control de Calidad
Mensajero(a) / Chofer
Ejecutivo(a) de Ventas
Supervisor(a) Comercial - Ultracréditos - La Paz
Analista de Inteligencia de Negocios
Ejecutivo(a) Comercial
Vendedor(a) (Tarde/Noche)
Asistente

In [14]:
len(Results)

918

In [15]:
import csv
# Open the file in the mode append and the encoding utf-8
outfile = open("/Users/Roberto/Documents/Python Scripts/Machine Learning Projects/Labor Market/TRABAJOPOLIS_FEB2019.csv", 'a',newline='', encoding = 'utf-8')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        # Transform the list shape:
        if Results[i][0] is not None:
            
            for cell in Results[i][0]:
                csvRow.append(cell)
                
            csvRow.append(Results[i][1])
            csvRow.append(Results[i][2])

            writer.writerow(csvRow)
finally:
    outfile.close() 

In [16]:
Results[400]

(['Muebles INTI',
  '786032',
  'Santa Cruz de la Sierra, Bolivia',
  'Administración y Oficina ',
  'De Bs. 2.122 a Bs. 2.999 ',
  '21 Agosto 2019',
  '20 Septiembre 2019',
  'se necesitaauxiliar contablepara realizar las siguientes funciones actividades de recepcion entrega y custodia de dinero en efectivo y cheques facturacion elaboracion de informesrequisitos saber trabajar en equipo bajo presion y proactivo con disposicion inmediata para el cargo se valorara la experiencia en el cargo y los conocimientos en todo el paquete officelas personas interesadas que cumplan con el perfil podran remitir su curriculum vitae indicando su pretension salarial por trabajopolisbo'],
 'Auxiliar Contable',
 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/786032/Auxiliar-Contable.html?searchId=1546400657&page=1')

In [17]:
Results[0]

(['HOTEL PALACIO DE SAL',
  '791062',
  'Otra Ciudad de Bolivia, Bolivia',
  'Turismo y Hotelería ',
  'No Declarado ',
  '02 Octubre 2019',
  '02 Octubre 2019',
  'el hotel jardines de uyuni con ubicacion en la ciudad de uyuni departamento de potosi busca incorporar a su equipo de trabajo un profesional para el siguiente puestoresponsable de recursos humanosrequisitos lic yo tecnico superior en administracion de empresas contabilidad o ramas afines al cargo conocimiento de administracion de personal planillas salariales beneficios sociales etcexperiencia experiencia laboral en rrhh y elaboracion de planillas y formularios de aportes a afp y cns tramites en general en el ministerio de trabajo elaboracion de finiquitos llenado de formularios planillas mensuales y subsidio dominio de excel avanzadocompetencias orientacion a resultados comunicacion efectiva planificacion y organizacion empatialas personas interesadas por favor enviar su hoja de vida indicando su pretension salarial por tr

In [18]:
Results = trabajopolis(['cajero'])

The request response cycle was successful
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/758480/Cajero(a)---La-Paz.html?searchId=1558925449.56&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/758322/Cajero(a)---Santa-Cruz.html?searchId=1558925449.56&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/757415/Auxiliar-de-Caja---Tarija.html?searchId=1558925449.56&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/757249/Cajero(a)-Plazo-Fijo---Riberalta.html?searchId=1558925449.56&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/756544/Cajero---Santa-Cruz.html?searchId=1558925449.56&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/756491/Cajero(a)---Oruro.html?searchId=1558925449.56&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/756019/Cajero(a)---Norte-Integrado.html?searchId=1558925449.56&page=1
https://www.trabajopolis.bo/oferta-

NameError: name 'time' is not defined

In [ ]:
hola = Results[1000:]

In [ ]:
for i in range(len(hola)-1):
    csvRow = []
        
    if Results[i][0] is not None:
            
        for cell in hola[i][0]:
            csvRow.append(cell)
                
        csvRow.append(hola[i][1])
        csvRow.append(hola[i][2])

print(csvRow)

In [53]:
csvrow = []
for cell in Results[i][0]:
    #print(cell)
    csvrow.append(cell)
csvrow.append(Results[i][1])
csvrow.append(Results[i][2])
print(csvrow)

['Guayabochi srl', '699870', 'Santa Cruz de la Sierra, Bolivia', 'Administración y Oficina ', 'No Declarado ', '07 Enero 2019', '06 Febrero 2019', 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo', 'Mensajero(a)', 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/699870/Mensajero(a).html?searchId=1546400657&page=1']


In [43]:
Results[i][0]

['Guayabochi srl',
 '699870',
 'Santa Cruz de la Sierra, Bolivia',
 'Administración y Oficina ',
 'No Declarado ',
 '07 Enero 2019',
 '06 Febrero 2019',
 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo']

In [44]:
for cell in Results[i][0]:
    print(cell)

Guayabochi srl
699870
Santa Cruz de la Sierra, Bolivia
Administración y Oficina 
No Declarado 
07 Enero 2019
06 Febrero 2019
empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo


In [45]:
csvRow.append(cell)